In [18]:
import pandas as pd

import nltk
import re
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from autocorrect import Speller

In [19]:
 #if you already downloaded it, comment it
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

#congfig things
stop_words = set(stopwords.words('english'))
words = set([w.lower() for w in nltk.corpus.words.words()])
spell = Speller(lang='en')
regex = re.compile('[^a-zA-Z0-9 ]')
ps = PorterStemmer()

min_len = 3
max_len = 999

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Harry\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Harry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Harry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
#reading data
xy_train_df = pd.read_csv('news_dataset.csv').dropna()

In [21]:
xy_train_df = xy_train_df.reset_index()

In [22]:
#additional preprocessing beside vocab things
def all_preprocessing(df,remove_stop=True,word_stem=True):
    temp = df.copy()
    for i in tqdm(range(len(temp))):
        text = regex.sub("",temp.loc[i].content).lower() #remove all non-numeric.space,non-alpha and ^ thing
        text_tokens = word_tokenize(text)
        if len(text_tokens) > max_len or len(text_tokens) < min_len: # if a text have less then min_len or more than max_len drop it
            temp.drop([i],inplace=True)
        else:
            #spell correcting
            for token_idx in range(len(text_tokens)):
                token = text_tokens[token_idx]
                if token not in words:
                    spell_check = spell(token)
                    token = spell_check
                    text_tokens[token_idx] = spell_check
            #remove stop words
            if remove_stop:
                text_tokens = [w for w in text_tokens if not w in stop_words]
            #word stemming
            if word_stem:
                text_tokens = [ps.stem(w) for w in text_tokens]
            temp.at[i,"content"] = " ".join(text_tokens)[0:]
    return temp

In [23]:
#additional preprocessing beside vocab things
def all_preprocessing_title(df,remove_stop=True,word_stem=True):
    temp = df.copy()
    for i in tqdm(range(len(temp))):
        text = regex.sub("",temp.loc[i].title).lower() #remove all non-numeric.space,non-alpha and ^ thing
        text_tokens = word_tokenize(text)
        if len(text_tokens) > max_len or len(text_tokens) < min_len: # if a text have less then min_len or more than max_len drop it
            temp.drop([i],inplace=True)
        else:
            #spell correcting
            for token_idx in range(len(text_tokens)):
                token = text_tokens[token_idx]
                if token not in words:
                    spell_check = spell(token)
                    token = spell_check
                    text_tokens[token_idx] = spell_check
            #remove stop words
            if remove_stop:
                text_tokens = [w for w in text_tokens if not w in stop_words]
            #word stemming
            if word_stem:
                text_tokens = [ps.stem(w) for w in text_tokens]
            temp.at[i,"title"] = " ".join(text_tokens)[0:]
    return temp

In [24]:
xy_train_preprocessed = all_preprocessing(xy_train_df)

  0%|          | 0/27985 [00:00<?, ?it/s]


NameError: name 'max_len' is not defined

In [51]:
len(xy_train_preprocessed)

20706

In [52]:
#the preprocess time is so long, save the result incase of something bad happens
xy_train_preprocessed.to_csv("news_dataset_preprocessed.csv")

In [54]:
xy_train_preprocessed = xy_train_preprocessed.reset_index()

In [55]:
xy_train_preprocessed_all = all_preprocessing_title(xy_train_preprocessed)

100%|████████████████████████████████████████████████████████████████████████████| 20706/20706 [11:13<00:00, 30.74it/s]


In [56]:
#the preprocess time is so long, save the result incase of something bad happens
xy_train_preprocessed_all.to_csv("news_dataset_preprocessed_all.csv")

In [57]:
len(xy_train_preprocessed_all)

20408